In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/glove840b300dtxt/glove.840B.300d.txt
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test_labels.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train-processed-seqlen128.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-unintended-bias-train.csv
/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train-processed-seqlen128.csv


In [5]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, SimpleRNN, Dense, Activation, Dropout, Embedding, BatchNormalization, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.callbacks import EarlyStopping

# Your code goes here





In [6]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2023-12-15 21:59:23.422375: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 21:59:23.422485: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 21:59:23.422561: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 21:59:23.422634: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 21:59:23.422700: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 21:59:23.422903: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2023-12-15 21:59:23.423041: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS:  8


In [7]:
train = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv')
validation = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')

In [8]:
train.drop(['severe_toxic','obscene','threat','insult','identity_hate'],axis=1,inplace=True)

In [9]:
train.head()

,id,comment_text,toxic
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0


In [10]:
#max no of word present in a comment
train['comment_text'].apply(lambda x : len(str(x).split())).max()

2321

In [11]:
x_train,x_valid,y_train,y_valid = train_test_split(train.comment_text.values,train.toxic.values,stratify=train.toxic.values,random_state=2,test_size=0.2,shuffle=True)

In [12]:
#Auc Score for Validation
def roc_auc(pred,target):
    fpr,tpr,th = metrics.roc_curve(target,pred)
    roc_auc=metrics.auc(fpr,tpr)
    return roc_auc

In [13]:
#tokenize 
token = text.Tokenizer(num_words=None)
max_len=2400

token.fit_on_texts(list(x_train)+list(x_valid))
x_train_seq = token.texts_to_sequences(x_train)
x_valid_seq = token.texts_to_sequences(x_valid)

x_train_pd = sequence.pad_sequences(x_train_seq,maxlen=max_len)
x_valid_pd = sequence.pad_sequences(x_valid_seq,maxlen=max_len)

word_index = token.word_index

# Simple RNN 

In [51]:
%%time
with strategy.scope():
    
    model1 = Sequential()
    model1.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
    model1.add(SimpleRNN(100))
    model1.add(Dense(1, activation='sigmoid'))
    model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 2400, 300)         90077400  
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 100)               40100     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 90117601 (343.77 MB)
Trainable params: 90117601 (343.77 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
CPU times: user 2.88 s, sys: 6.99 s, total: 9.87 s
Wall time: 2.83 s


In [52]:
%%time 
model1.fit(x_train_pd,y_train,epochs=5,batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5


2023-12-15 22:47:10.681878: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


350/350 [==============================] - 51s 120ms/step - loss: 0.2652 - accuracy: 0.9066
Epoch 2/5
350/350 [==============================] - 37s 105ms/step - loss: 0.1619 - accuracy: 0.9435
Epoch 3/5
350/350 [==============================] - 37s 105ms/step - loss: 0.1542 - accuracy: 0.9468
Epoch 4/5
350/350 [==============================] - 37s 105ms/step - loss: 0.1249 - accuracy: 0.9542
Epoch 5/5
350/350 [==============================] - 37s 105ms/step - loss: 0.0778 - accuracy: 0.9706
CPU times: user 2min 31s, sys: 1min, total: 3min 31s
Wall time: 3min 22s


In [53]:
scores1 = model.predict(x_valid_pd)


1398/1398 [==============================] - 86s 61ms/step


In [54]:
print("Auc: %.2f%%" % (roc_auc(scores1,y_valid)))

Auc: 0.98%


In [55]:
scores_model = []
scores_model.append({'Model':'SimpleRNN','AUC_Score':roc_auc(scores1,y_valid)})

In [56]:
scores_model

[{'Model': 'SimpleRNN', 'AUC_Score': 0.9790781709898231}]

# LSTM 


In [23]:
embedding_index = {}
ff = open('/kaggle/input/glove840b300dtxt/glove.840B.300d.txt','r',encoding='utf-8')
for line in tqdm(ff):
    values = line.split(' ')
    word = values[0]
    coefs = np.asarray([float(val) for val in values[1:]])
    embedding_index[word] = coefs
ff.close()

print('Found %s word Vectors.'%len(embedding_index))

2196018it [02:52, 12722.61it/s]

Found 2196017 word Vectors.


In [26]:
embedding_matrix = np.zeros((len(word_index)+1,300))
for word,i in tqdm(word_index.items()):
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

100%|██████████| 300257/300257 [00:00<00:00, 331998.91it/s]


In [57]:
%%time 
with strategy.scope():
    model = Sequential()
    model.add(Embedding(len(word_index)+1,300,weights=[embedding_matrix],input_length=max_len,trainable=False))
    model.add(LSTM(100,dropout=0.3,recurrent_dropout=0.3))
    model.add(Dense(1,activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

CPU times: user 5.81 s, sys: 12.9 s, total: 18.7 s
Wall time: 8.53 s


In [58]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 2400, 300)         90077400  
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 90237901 (344.23 MB)
Trainable params: 160501 (626.96 KB)
Non-trainable params: 90077400 (343.62 MB)
_________________________________________________________________


In [59]:
%%time
model.fit(x_train_pd,y_train,epochs=5,batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5


2023-12-15 22:52:26.092756: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


350/350 [==============================] - 92s 241ms/step - loss: 0.1527 - accuracy: 0.9424
Epoch 2/5
350/350 [==============================] - 82s 233ms/step - loss: 0.1153 - accuracy: 0.9540
Epoch 3/5
350/350 [==============================] - 82s 233ms/step - loss: 0.1105 - accuracy: 0.9552
Epoch 4/5
350/350 [==============================] - 82s 233ms/step - loss: 0.1050 - accuracy: 0.9578
Epoch 5/5
350/350 [==============================] - 81s 233ms/step - loss: 0.1016 - accuracy: 0.9586
CPU times: user 2min 15s, sys: 50.5 s, total: 3min 5s
Wall time: 7min 1s


In [63]:
scores2 = model.predict(x_valid_pd)

1398/1398 [==============================] - 87s 62ms/step


In [64]:
print("Auc: %.2f%%" % (roc_auc(scores2,y_valid)))

Auc: 0.98%


In [65]:
scores_model.append({'Model':'LSTM','AUC_Score':roc_auc(scores2,y_valid)})

In [66]:
scores_model

[{'Model': 'SimpleRNN', 'AUC_Score': 0.9790781709898231},
 {'Model': 'LSTM', 'AUC_Score': 0.9786417912235196}]

# GRU 


In [70]:
%%time
with strategy.scope():
    model3 = Sequential()
    model3.add(Embedding(len(word_index)+1,300,weights=[embedding_matrix],input_length=max_len,trainable=False))
    model3.add(SpatialDropout1D(0.3))
    model3.add(GRU(300))
    model3.add(Dense(1,activation='sigmoid'))
    model3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

CPU times: user 11 s, sys: 12.8 s, total: 23.8 s
Wall time: 9.92 s


In [71]:
model3.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 2400, 300)         90077400  
                                                                 
 spatial_dropout1d (Spatial  (None, 2400, 300)         0         
 Dropout1D)                                                      
                                                                 
 gru (GRU)                   (None, 300)               541800    
                                                                 
 dense_4 (Dense)             (None, 1)                 301       
                                                                 
Total params: 90619501 (345.69 MB)
Trainable params: 542101 (2.07 MB)
Non-trainable params: 90077400 (343.62 MB)
_________________________________________________________________


In [72]:
model3.fit(x_train_pd,y_train,epochs=5,batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5


2023-12-15 23:10:46.120955: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


350/350 [==============================] - 95s 247ms/step - loss: 0.1437 - accuracy: 0.9449
Epoch 2/5
350/350 [==============================] - 84s 240ms/step - loss: 0.1112 - accuracy: 0.9550
Epoch 3/5
350/350 [==============================] - 84s 240ms/step - loss: 0.1058 - accuracy: 0.9572
Epoch 4/5
350/350 [==============================] - 84s 241ms/step - loss: 0.1013 - accuracy: 0.9585
Epoch 5/5
350/350 [==============================] - 84s 241ms/step - loss: 0.0968 - accuracy: 0.9603


In [76]:
scores3 = model3.predict(x_valid_pd)

1398/1398 [==============================] - 78s 55ms/step


In [77]:
print("Auc: %.2f%%" % (roc_auc(scores3,y_valid)))

Auc: 0.98%


In [78]:
scores_model.append({'Model':'GRU','AUC_Score':roc_auc(scores3,y_valid)})

In [80]:
scores_model

[{'Model': 'SimpleRNN', 'AUC_Score': 0.9790781709898231},
 {'Model': 'LSTM', 'AUC_Score': 0.9786417912235196},
 {'Model': 'GRU', 'AUC_Score': 0.9803200034630966},
 {'Model': 'GRU', 'AUC_Score': 0.9803200034630966}]

# Bi-Directional RNN's

In [81]:
%%time
with strategy.scope():
    model4 = Sequential()
    model4.add(Embedding(len(word_index)+1,300,weights=[embedding_matrix],input_length=max_len,trainable=False))
    
    model4.add(Bidirectional(LSTM(300, dropout=0.3, recurrent_dropout=0.3)))
    model4.add(Dense(1,activation='sigmoid'))
    model4.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

CPU times: user 11.3 s, sys: 13.3 s, total: 24.6 s
Wall time: 10.4 s


In [82]:
model4.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 2400, 300)         90077400  
                                                                 
 bidirectional (Bidirection  (None, 600)               1442400   
 al)                                                             
                                                                 
 dense_5 (Dense)             (None, 1)                 601       
                                                                 
Total params: 91520401 (349.12 MB)
Trainable params: 1443001 (5.50 MB)
Non-trainable params: 90077400 (343.62 MB)
_________________________________________________________________


In [83]:
model4.fit(x_train_pd,y_train,epochs=5,batch_size=64*strategy.num_replicas_in_sync)

Epoch 1/5


2023-12-15 23:25:33.852458: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


350/350 [==============================] - 319s 878ms/step - loss: 0.1488 - accuracy: 0.9440
Epoch 2/5
350/350 [==============================] - 302s 864ms/step - loss: 0.1151 - accuracy: 0.9538
Epoch 3/5
350/350 [==============================] - 302s 864ms/step - loss: 0.1080 - accuracy: 0.9564
Epoch 4/5
350/350 [==============================] - 302s 864ms/step - loss: 0.1075 - accuracy: 0.9572
Epoch 5/5
350/350 [==============================] - 302s 864ms/step - loss: 0.1011 - accuracy: 0.9588


In [84]:
scores4 = model4.predict(x_valid_pd)

2023-12-15 23:50:58.572384: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


1398/1398 [==============================] - 224s 159ms/step


In [85]:
print("Auc: %.2f%%" % (roc_auc(scores4,y_valid)))

Auc: 0.98%


In [86]:
scores_model.append({'Model':'Bi-Directional RNN','AUC_Score':roc_auc(scores4,y_valid)})

In [87]:
scores_model

[{'Model': 'SimpleRNN', 'AUC_Score': 0.9790781709898231},
 {'Model': 'LSTM', 'AUC_Score': 0.9786417912235196},
 {'Model': 'GRU', 'AUC_Score': 0.9803200034630966},
 {'Model': 'GRU', 'AUC_Score': 0.9803200034630966},
 {'Model': 'Bi-Directional RNN', 'AUC_Score': 0.9791065231841699}]

In [93]:
def remove_duplicates(data):
    # Convert the list of dictionaries to a list of tuples and then to a set to remove duplicates
    unique_data_set = set(tuple(d.items()) for d in data)

    # Convert the set of tuples back to a list of dictionaries
    unique_data_list = [dict(t) for t in unique_data_set]

    return unique_data_list

In [99]:
results = pd.DataFrame(scores_model).sort_values(by='AUC_Score',ascending=False)
results.drop_duplicates(inplace=True)
results.style.background_gradient(cmap='Blues')

,Model,AUC_Score
2,GRU,0.980320
4,Bi-Directional RNN,0.979107
0,SimpleRNN,0.979078
1,LSTM,0.978642


In [107]:

from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff

In [108]:
fig = go.Figure(go.Funnelarea(
    text =results.Model,
    values = results.AUC_Score,
    title = {"position": "top center", "text": "Funnel-Chart of Sentiment Distribution"}
    ))
fig.show()

# BertModel

In [109]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from kaggle_datasets import KaggleDatasets
import transformers

from tokenizers import BertWordPieceTokenizer

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [110]:
train1 = pd.read_csv("/kaggle/input/jigsaw-multilingual-toxic-comment-classification/jigsaw-toxic-comment-train.csv")
valid = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/validation.csv')
test = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/test.csv')
sub = pd.read_csv('/kaggle/input/jigsaw-multilingual-toxic-comment-classification/sample_submission.csv')

In [111]:
test.head()

,id,content,lang
0,0,Doctor Who adlı viki başlığına 12. doctor olar...,tr
1,1,"Вполне возможно, но я пока не вижу необходимо...",ru
2,2,"Quindi tu sei uno di quelli conservativi , ...",it
3,3,Malesef gerçekleştirilmedi ancak şöyle bir şey...,tr
4,4,:Resim:Seldabagcan.jpg resminde kaynak sorunu ...,tr


In [112]:
sub.head()

,id,toxic
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [113]:
train1.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [122]:
def encode(text,tokenizer,chunk_size=256,maxlen=512):
    tokenizer.enable_truncation(max_length=maxlen)
    tokenizer.enable_padding(length=maxlen)
    all_ids=[]
    
    for i in tqdm(range(0,len(text),chunk_size)):
        text_chunk = text[i:i+chunk_size].tolist()
        encs = tokenizer.encode_batch(text_chunk)
        all_ids.extend([enc.ids for enc in encs])
    
    return np.array(all_ids)

In [120]:
AUTO = tf.data.experimental.AUTOTUNE
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 192


In [116]:
tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
tokenizer.save_pretrained('.')
fast_tokenizer = BertWordPieceTokenizer('vocab.txt', lowercase=False)
fast_tokenizer

tokenizer_config.json: 100%|██████████| 29.0/29.0 [00:00<00:00, 97.5kB/s]
vocab.txt: 100%|██████████| 996k/996k [00:00<00:00, 11.1MB/s]
tokenizer.json: 100%|██████████| 1.96M/1.96M [00:00<00:00, 21.0MB/s]
config.json: 100%|██████████| 466/466 [00:00<00:00, 2.17MB/s]


Tokenizer(vocabulary_size=119547, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=False, wordpieces_prefix=##)

In [123]:
x_train = encode(train1.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_valid = encode(valid.comment_text.astype(str), fast_tokenizer, maxlen=MAX_LEN)
x_test = encode(test.content.astype(str), fast_tokenizer, maxlen=MAX_LEN)

y_train = train1.toxic.values
y_valid = valid.toxic.values

100%|██████████| 250/250 [00:03<00:00, 67.46it/s]


In [134]:
train1.comment_text[1]

"D'aww! He matches this background colour I'm seemingly stuck with. Thanks.  (talk) 21:51, January 11, 2016 (UTC)"

In [154]:
tt = fast_tokenizer.encode_batch([train1.comment_text[1]])
tt

[Encoding(num_tokens=192, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [155]:
x_train[:1].shape

(1, 192)

In [159]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_train, y_train))
    .repeat()
    .shuffle(2048)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((x_valid, y_valid))
    .batch(BATCH_SIZE)
    .cache()
    .prefetch(AUTO)
)
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(x_test)
    .batch(BATCH_SIZE)
)

In [165]:
from transformers import DistilBertModel


In [170]:
def MODEL(tranformer,max_len=512):
    input_word_ids = Input(shape=(max_len,),dtype=tf.int32,name='input_word_ids')
    sq_output = tranformer(input_word_ids)[0]
    cls_token = sq_output[:,0,:]
    out = Dense(1,activation='sigmoid')(cls_token)
    
    model5 = Model(inputs=input_word_ids,outputs=out)
    model5.compile(Adam(lr=1e-5),loss='binary_crossentropy',metrics=['accuracy'])
    
    return model5

In [171]:
%%time 
with strategy.scope():
    transformer_layer = (
        transformers.TFDistilBertModel.from_pretrained('distilbert-base-multilingual-cased')
    )
    
    model5 = MODEL(transformer_layer,max_len=MAX_LEN)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


CPU times: user 10.4 s, sys: 16.6 s, total: 27 s
Wall time: 13.2 s


In [172]:
model5.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_word_ids (InputLayer  [(None, 192)]             0         
 )                                                               
                                                                 
 tf_distil_bert_model_5 (TF  TFBaseModelOutput(last_   134734080 
 DistilBertModel)            hidden_state=(None, 192             
                             , 768),                             
                              hidden_states=None, at             
                             tentions=None)                      
                                                                 
 tf.__operators__.getitem_1  (None, 768)               0         
  (SlicingOpLambda)                                              
                                                                 
 dense_7 (Dense)             (None, 1)                 769 

In [181]:
%%time
n_steps = x_train.shape[0] // BATCH_SIZE
train_history = model5.fit(
    train_dataset,
    steps_per_epoch=n_steps,
    validation_data=valid_dataset,
    epochs=EPOCHS
)

Epoch 1/3


2023-12-16 01:00:49.893601: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


1746/1746 [==============================] - ETA: 0s - loss: 0.3329 - accuracy: 0.9025

2023-12-16 01:03:53.418759: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


1746/1746 [==============================] - 208s 91ms/step - loss: 0.3329 - accuracy: 0.9025 - val_loss: 0.4406 - val_accuracy: 0.8462
Epoch 2/3
1746/1746 [==============================] - 149s 85ms/step - loss: 0.3168 - accuracy: 0.9044 - val_loss: 0.4511 - val_accuracy: 0.8462
Epoch 3/3
1746/1746 [==============================] - 148s 85ms/step - loss: 0.3166 - accuracy: 0.9043 - val_loss: 0.4564 - val_accuracy: 0.8462
CPU times: user 15min 12s, sys: 1min 39s, total: 16min 52s
Wall time: 8min 26s


In [182]:
n_steps = x_valid.shape[0] // BATCH_SIZE
train_history_2 = model5.fit(
    valid_dataset.repeat(),
    steps_per_epoch=n_steps,
    epochs=EPOCHS*2
)

Epoch 1/6
62/62 [==============================] - 5s 84ms/step - loss: 0.4335 - accuracy: 0.8459
Epoch 2/6
62/62 [==============================] - 37s 85ms/step - loss: 0.4304 - accuracy: 0.8462
Epoch 3/6
62/62 [==============================] - 5s 83ms/step - loss: 0.4309 - accuracy: 0.8460
Epoch 4/6
62/62 [==============================] - 5s 84ms/step - loss: 0.4295 - accuracy: 0.8468
Epoch 5/6
62/62 [==============================] - 5s 83ms/step - loss: 0.4324 - accuracy: 0.8450
Epoch 6/6
62/62 [==============================] - 5s 83ms/step - loss: 0.4319 - accuracy: 0.8454


In [183]:
score5 = model5.predict(valid_dataset)

2023-12-16 01:10:54.237264: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


63/63 [==============================] - 11s 77ms/step


In [191]:
score5

array([[0.15128443],
       [0.151285  ],
       [0.15128466],
       ...,
       [0.1512846 ],
       [0.15128413],
       [0.15128458]], dtype=float32)

In [187]:
score6 = model5.predict(test_dataset,verbose=1)

2023-12-16 01:12:44.567869: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


499/499 [==============================] - 20s 27ms/step


In [193]:
sub['toxic'] = score6

In [195]:
sub.to_csv('submission.csv',index=False)